# A/B Testing

This notebook visualizes and measures the results of an A/B test.

1. Measure weekly conversions from visiting the landing page to signing up.

undefined. Measure weekly retention of users who signed up.

In [1]:


visits = _deepnote_execute_sql("""select
    visits.user_id,
    visits.visited_at,
    users.signed_up_at,
    users_ab.variant
from
    visits
    left join users on visits.user_id = users.user_id
    inner join users_ab on visits.user_id = users_ab.user_id

    
""", 'SQL_FDA30E6F_BF07_4968_8270_DC37A349155E')
visits

,user_id,visited_at,signed_up_at,variant
0,ff8cb87a82494cb3b9df975d62de1bc1,2021-08-06 00:00:00+00:00,NaT,variant_a
1,ae40ee41d194447b9a07c63de65e0ba6,2021-06-28 00:00:00+00:00,NaT,variant_b
2,6a9d620fa953498caf45d441e8c6efa8,2021-09-22 00:00:00+00:00,NaT,variant_b
3,63f0d3c6e6d64ddf870697264be0b84d,2021-06-24 00:00:00+00:00,NaT,variant_a
4,679a0784184041718db48f8d5d26d263,2021-08-30 00:00:00+00:00,NaT,variant_b
...,...,...,...,...
83681,4f7660c3932340c7a332cb9f7e3a01c3,2021-09-28 00:00:00+00:00,2021-10-08 00:00:00+00:00,variant_b
83682,932140cc307949a684807335b85252f4,2021-07-09 00:00:00+00:00,NaT,variant_a
83683,4a983ff750e24f13845d01afaf2eb68f,2021-09-15 00:00:00+00:00,NaT,variant_b
83684,fb9da17af5d24683a1c26cc88dcf74fa,2021-07-24 00:00:00+00:00,NaT,variant_a


In [2]:
visits = visits.sort_values('visited_at').drop_duplicates('user_id',keep='first').copy()

visits

,user_id,visited_at,signed_up_at,variant
19903,5f84735855ed4b0c94523e2a2617ccb4,2021-06-11 00:00:00+00:00,NaT,variant_a
42514,92c0cfb58c974b12ab601338e33175d7,2021-06-11 00:00:00+00:00,NaT,variant_b
9049,5e3c78569ddd44a28ed81f5421be50fc,2021-06-11 00:00:00+00:00,NaT,variant_a
32752,195d65944f3f44ec8a691ed9dd1e3a30,2021-06-11 00:00:00+00:00,NaT,variant_a
34971,09a6505498564a9e958ce54729564bae,2021-06-11 00:00:00+00:00,NaT,variant_a
...,...,...,...,...
50973,b05ef05dc6ba4b1ba6c757f5654117c9,2021-09-30 00:00:00+00:00,NaT,variant_a
55181,76d64e5a1b1f4648b7d8637acec8d850,2021-09-30 00:00:00+00:00,NaT,variant_b
77218,2ff8bd6037954a7a9b2305f7d262e56d,2021-09-30 00:00:00+00:00,NaT,variant_a
80756,abc95de891014e0285122e6dd424eaa9,2021-09-30 00:00:00+00:00,NaT,variant_b


We need to check whether one variant resulted in a higher conversion rate than another. Let's start with a time series.

In [12]:
visits['registered'] = visits.signed_up_at.notna()
visits = visits.sort_values('visited_at').drop_duplicates('user_id',keep='first').copy()

conversion = visits.copy()
conversion['week'] = conversion.visited_at.dt.tz_localize(None).dt.to_period('W').dt.to_timestamp()
conversion = conversion.groupby(['variant','week']).registered.value_counts(normalize=True,dropna=False).reset_index(name='conversion')
conversion = conversion.loc[conversion.registered == True]

In [13]:
DeepnoteChart(conversion, """{"data":{"name":"placeholder"},"mark":{"type":"line","tooltip":{"content":"data"}},"height":440,"$schema":"https://vega.github.io/schema/vega-lite/v4.json","autosize":{"type":"fit"},"encoding":{"x":{"sort":null,"type":"temporal","field":"week","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"conversion","scale":{"type":"linear","zero":false}},"color":{"sort":null,"type":"nominal","field":"variant","scale":{"type":"linear","zero":false}}},"width":"container","config":{"legend":{"labelFont":"Inter, sans-serif","labelFontSize":12,"titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold"},"title":{"anchor":"start","color":"#000000","font":"Inter, sans-serif","fontSize":22,"fontWeight":"normal"},"axis":{"labelFont":"Inter, sans-serif","labelFontSize":12,"labelFontWeight":"normal","titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold","labelOverlap":"greedy"}}}""")

Here is the overall conversion rate for each variant:

In [5]:
conversion.groupby('variant').conversion.mean().reset_index(name='conversion_rate')

,variant,conversion_rate
0,variant_a,0.008130
1,variant_b,0.013527


### Significance Test

It looks like Variant B has a higher conversion rate than Variant A. We need to make sure these results are significant.

In [14]:
from scipy.stats import chi2_contingency

# Calculate the proportion of registered users for variant A vs B
proportions = visits.groupby('variant').registered.value_counts(dropna=False).reset_index(name='nusers')
proportions = proportions.pivot(index='variant',columns='registered',values='nusers')

# Chi-Square test
g,p,dof,expctd = chi2_contingency(proportions)

print(f'p-value is {p}')
if p < 0.05:
    print(f'The difference in Conversion Rate between Variants A and B is statistically significant.')
else:
    print(f'The difference in Conversion Rate between Variants A and B is NOT statistically significant.')

p-value is 4.536508742253485e-12
The difference in Conversion Rate between Variants A and B is statistically significant.


## Retention Rate

We also want to know whether one variant of the website results in higher retention than another variant. For this, let's look at the weekly sessions of our users.

In [7]:


sessions_weekly = _deepnote_execute_sql("""select
    sessions.user_id,
    date_trunc('week',users.signed_up_at) as signed_up_at_week,
    floor(extract('day' from session_started_at - signed_up_at)/7) as week, -- The number of weeks that passed since the user signed up
    users_ab.variant
from
    sessions
    left join users on sessions.user_id = users.user_id
    inner join users_ab on sessions.user_id = users_ab.user_id
""", 'SQL_FDA30E6F_BF07_4968_8270_DC37A349155E')
sessions_weekly

,user_id,signed_up_at_week,week,variant
0,20730e99d00d466eb218cfecee852647,2021-09-27 00:00:00+00:00,4.0,variant_b
1,5541a58e2e1f4cbdafc9c1b12535f2d9,2021-07-12 00:00:00+00:00,14.0,variant_b
2,5541a58e2e1f4cbdafc9c1b12535f2d9,2021-07-12 00:00:00+00:00,8.0,variant_b
3,5541a58e2e1f4cbdafc9c1b12535f2d9,2021-07-12 00:00:00+00:00,8.0,variant_b
4,5541a58e2e1f4cbdafc9c1b12535f2d9,2021-07-12 00:00:00+00:00,8.0,variant_b
...,...,...,...,...
14240,b6d3a798e7a24dfc8ee996817824d101,2021-07-05 00:00:00+00:00,9.0,variant_b
14241,2c3fede222b54a4e8c6181a42ed8c3a1,2021-08-16 00:00:00+00:00,5.0,variant_b
14242,afe8126a93a2473e8458b77a911344fc,2021-08-30 00:00:00+00:00,4.0,variant_b
14243,cbd0ebd955014ee8a6f879ff5aaf166f,2021-07-26 00:00:00+00:00,8.0,variant_b


In [15]:
import pandas as pd

def get_retention(df):
    retention = df.copy()

    # Save the cohort size before we start calculating retention
    cohort_size = retention.groupby(['signed_up_at_week']).user_id.nunique().reset_index(name='cohort_size')

    # For each cohort-week, calculate the number of users who visited
    retention = retention.groupby(['signed_up_at_week','week']).user_id.nunique().reset_index(name='n_users')

    # Pivot and melt the table. This is a little trick that allows us to add rows during weeks where a cohort was not active.
    retention = retention.pivot(index=['signed_up_at_week'],columns='week',values='n_users').fillna(0)
    retention = retention.melt(value_name='n_users',ignore_index=False).reset_index()

    # If part of the cohort is still not finished the week, then exclude that cohort-week from the data
    # To do this we add 6 days to the sign up date. This gives us the last sign ups of that cohort.
    retention = retention.loc[
        ~(retention.signed_up_at_week + pd.to_timedelta(retention.week + 1,'W') + pd.Timedelta(6,'D') >
        pd.Timestamp.now(tz='UTC').floor('D'))
    ]

    # Divide by the cohort size to get a percentage
    retention = retention.merge(cohort_size,on=['signed_up_at_week'])
    retention['prop'] = retention.n_users / retention.cohort_size

    retention_avg = retention.groupby('week').prop.mean().reset_index()

    retention_avg = retention_avg.rename(columns = dict(
        prop='Retention',
        week='Week',
    ))

    return retention_avg

sessions_a = sessions_weekly.loc[sessions_weekly.variant == 'variant_a']
retention_a = get_retention(sessions_a)
retention_a['variant'] = 'variant_a'

sessions_b = sessions_weekly.loc[sessions_weekly.variant == 'variant_b']
retention_b = get_retention(sessions_b)
retention_b['variant'] = 'variant_b'

retention = pd.concat([retention_a,retention_b])

In [16]:
DeepnoteChart(retention, """{"data":{"name":"placeholder"},"mark":{"type":"line","tooltip":{"content":"data"}},"height":440,"$schema":"https://vega.github.io/schema/vega-lite/v4.json","autosize":{"type":"fit"},"encoding":{"x":{"sort":null,"type":"quantitative","field":"Week","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"Retention","scale":{"type":"linear","zero":false}},"color":{"sort":null,"type":"nominal","field":"variant","scale":{"type":"linear","zero":false}}},"width":"container","config":{"legend":{"labelFont":"Inter, sans-serif","labelFontSize":12,"titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold"},"title":{"anchor":"start","color":"#000000","font":"Inter, sans-serif","fontSize":22,"fontWeight":"normal"},"axis":{"labelFont":"Inter, sans-serif","labelFontSize":12,"labelFontWeight":"normal","titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold","labelOverlap":"greedy"}}}""")

Here is the Week-4 Retention for each variant.

In [10]:
retention.loc[retention.Week == 4][['variant','Retention']].reset_index(drop=True)

,variant,Retention
0,variant_a,0.176021
1,variant_b,0.377029


### Significance Test

Once again, check that the difference in retention is significance.

In [11]:
from scipy.stats import ttest_ind

# Label all users who retained 4+ weeks later
users_retention = sessions_weekly.groupby(['user_id','variant']).apply(lambda x: x.week.max() >= 4).reset_index(name='retained')

# Calculate the proportion of retained users for variant A vs B
proportions = users_retention.groupby('variant').retained.value_counts(dropna=False).reset_index(name='nusers')
proportions = proportions.pivot(index='variant',columns='retained',values='nusers')

# Chi-Square test
g,p,dof,expctd = chi2_contingency(proportions)

print(f'p-value is {p}')
if p < 0.05:
    print(f'The difference in Retention between Variants A and B is statistically significant.')
else:
    print(f'The difference in Retention between Variants A and B is NOT statistically significant.')

p-value is 2.5434010296902054e-12
The difference in Retention between Variants A and B is statistically significant.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=46cbc318-b565-4072-ad7f-3afdddf5ea13' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>